In [1]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
from scipy.stats import linregress

### Determine slope and intercept for McMillan flow sensors based on factory calibration data

[Values for sensor 46747](/files/flow_sensor_46747_calibration.JPG)

[Values for sensor 47499](/files/flow_sensor_47499_calibration.JPG)

These values will go directly into the Arduino code and there shouldn't be a need to use them during data workup, as the Arduino will report flow in mL/min.

In [2]:
#SN 46747
flow_100_3 = pd.DataFrame(
    {
        'flow':[20, 50, 100],
        'voltage':[0.90, 2.64, 4.90]
    }
)
#SN 47499
flow_100_5 = pd.DataFrame(
    {
        'flow':[100, 250, 500],
        'voltage':[0.90, 2.55, 5.05]
    }
)
cal_100_3 = linregress(flow_100_3['voltage'], flow_100_3['flow'])
cal_100_5 = linregress(flow_100_5['voltage'], flow_100_5['flow'])
print(f'100-3: Slope: {cal_100_3.slope}, Intercept: {cal_100_3.intercept}')
print(f'100-5: Slope: {cal_100_5.slope}, Intercept: {cal_100_5.intercept}')

100-3: Slope: 20.103417415227547, Intercept: 0.10905233849315721
100-5: Slope: 96.6787554876885, Intercept: 9.410192784882554


### Calibrate Apogee SO-110 moisture sensors

See [Apogee SO-110 manual](/files/SO-100-200-manual.pdf) for details.

Alternatively, the spreadsheet [O2-Readings-Calculator.xlsx](/files/O2-Readings-Calculator.xlsx) is directly from Apogee and can be used for the same purpose.

In [3]:
file_descriptions = Path(".") / "files" / "calibration_files_description.txt"
cal_files = {}
with open(file_descriptions, "r") as f:
    for line in f:
        line_ = line.split(";")
        print(line_)

['RXN00009.CSV', ' dry air', ' Sensors in sealed container with air and Drierite', '\n']
['RXN00010.CSV', ' atmospheric', ' Sensors open to atmosphere', '\n']
['RXN00011.CSV', ' wet air', ' Sensors in sealed container with water in bottom', '\n']
['RXN00012.CSV', ' dry N2 no humidity sensor', ' Note: this had the humidity sensor unattached. Use RXN00016 instead', '\n']
['RXN00015.CSV', ' wet N2', ' N2 stream through bubbler, then the two oxygen sensors, then the BME280 sensor', '\n']
['RXN00016.CSV', ' dry N2', ' N2 stream straight to oxygen sensors then the BME280 sesnor', '']


In [4]:
dry_air = Path(".") / "files" / "RXN00009.CSV"
dry_n2 = Path(".") / "files" / "RXN00016.CSV"
wet_air = Path(".") / "files" / "RXN00011.CSV"
wet_n2 = Path(".") / "files" / "RXN00015.CSV"

In [5]:
class Calibration:
    def __init__(self, air_path: Path, n2_path: Path, selected_samples: int = 200):
        self.full_air_df = pd.read_csv(air_path, header=0, skiprows=4)
        self.full_n2_df = pd.read_csv(n2_path, header=0, skiprows=4)
        self.air_df = self._selected_df(self.full_air_df, selected_samples)
        self.n2_df = self._selected_df(self.full_n2_df, selected_samples)
        self.oxy1_Vc = np.mean(self.air_df["oxy1_voltage (V)"])
        self.oxy2_Vc = np.mean(self.air_df["oxy2_voltage (V)"])
        self.oxy1_V0 = np.mean(self.n2_df["oxy1_voltage (V)"])
        self.oxy2_V0 = np.mean(self.n2_df["oxy2_voltage (V)"])
        self.oxy1_Tc = np.mean(self.air_df["oxy1_temp (C)"])
        self.oxy2_Tc = np.mean(self.air_df["oxy2_temp (C)"])
        self.Pc = np.mean(self.air_df["pressure (kPa)"])
        self.RH = np.mean(self.air_df["humidity (%)"])
        self.oxy1_calibration_factor = (0.2095 * self.Pc) / (self.oxy1_Vc - self.oxy1_V0)
        self.oxy2_calibration_factor = (0.2095 * self.Pc) / (self.oxy2_Vc - self.oxy2_V0)
        self.c1 = -0.06949
        self.c2 = 0.001422
        self.c3 = -0.0000008213

    def _selected_df(self, df: pd.DataFrame, n: int):
        df = df.iloc[-n:].copy()
        df.reset_index(drop=True, inplace=True)
        return df

    def display(self):
        print(f"Oxy1 Vc:\t{self.oxy1_Vc}")
        print(f"Oxy2 Vc:\t{self.oxy2_Vc}")
        print(f"Oxy1 V0:\t{self.oxy1_V0}")
        print(f"Oxy2 V0:\t{self.oxy2_V0}")
        print(f"Oxy1 Tc:\t{self.oxy1_Tc}")
        print(f"Oxy2 Tc:\t{self.oxy2_Tc}")
        print(f"Pc:\t\t{self.Pc}")
        print(f"RH:\t\t{self.RH}")
        print(f"Oxy1 CF:\t{self.oxy1_calibration_factor}")
        print(f"Oxy2 CF:\t{self.oxy2_calibration_factor}")

    def as_dict(self):
        return {
            "oxy1": {
                "Vc": self.oxy1_Vc,
                "V0": self.oxy1_V0,
                "Pc": self.Pc,
                "RH": self.RH,
                "Tc": self.oxy1_Tc,
                "calibration_factor": self.oxy1_calibration_factor,
            },
            "oxy2": {
                "Vc": self.oxy2_Vc,
                "V0": self.oxy2_V0,
                "Pc": self.Pc,
                "RH": self.RH,
                "Tc": self.oxy2_Tc,
                "calibration_factor": self.oxy2_calibration_factor,
            },
        }



In [6]:
dry = Calibration(dry_air, dry_n2)
wet = Calibration(wet_air, wet_n2)

### Dry Calibration Values

In [7]:
dry.display()

Oxy1 Vc:	0.052950695000000006
Oxy2 Vc:	0.05059791000000002
Oxy1 V0:	0.0014562249999999998
Oxy2 V0:	0.00150478
Oxy1 Tc:	22.025949999999998
Oxy2 Tc:	22.699700000000004
Pc:		99.99825000000001
RH:		4.75015
Oxy1 CF:	406.83268271330877
Oxy2 CF:	426.73248527848995


### Wet Calibration Values

In [8]:
wet.display()

Oxy1 Vc:	0.05166569000000001
Oxy2 Vc:	0.048979350000000005
Oxy1 V0:	0.001474925
Oxy2 V0:	0.0015370499999999999
Oxy1 Tc:	22.521099999999997
Oxy2 Tc:	23.3594
Pc:		100.04404999999997
RH:		100.0
Oxy1 CF:	417.5913332861133
Oxy2 CF:	441.78356603705953


In [9]:
calibration_json = Path(".") / "calibration_values.json"
with open(calibration_json, "w") as f:
    json.dump(
        {"dry": dry.as_dict(), "wet": wet.as_dict()}, f, indent=4, sort_keys=True
    )

### Arduino Values

The Arduino code uses the dry calibration values for OXY1 (to be used pre-reaction with gas straight from the cylinder(s)) and the wet calibration values for OXY2 (post-reaction). If your reaction is not in water and you need accurate post-reaction values you should use the raw voltages from the sensor and convert to %O2 using the appropriate calibration values.